<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Ames Housing Data

_Author: Sapna Srinivas_


## Importing libaries
---

We'll need the following libraries for today's lesson:

1. `pandas`
2. `numpy`
3. `seaborn`
4. `matplotlib.pyplot`
4. `train_test_split` and `cross_val_score` from `sklearn`'s `model_selection` module
5. `LinearRegression`, `LassoCV` and `RidgeCV` from `sklearn`'s `linear_model` module
6. `StandardScaler` from `sklearn`'s `preprocessing` module
7. `r2_score` from `sklearn`'s `metrics` module 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score

%matplotlib inline

## Load the Data 

The test dataset has all of those columns except for the target that you are trying to predict in your Regression model

In [4]:
test = pd.read_csv('./datasets/test.csv')

## To display all the rows and columns of a dataframe

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
test.head()


,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,Stone,Fa,TA,No,Unf,0,Unf,0,1020,1020,GasA,Gd,N,FuseP,908,1020,0,1928,0,0,2,0,4,2,Fa,9,Typ,0,NaN,Detchd,1910.0,Unf,1,440,Po,Po,Y,0,60,112,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,Unf,0,Unf,0,1967,1967,GasA,TA,Y,SBrkr,1967,0,0,1967,0,0,2,0,6,2,TA,10,Typ,0,NaN,Attchd,1977.0,Fin,2,580,TA,TA,Y,170,0,0,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,Gd,Av,GLQ,554,Unf,0,100,654,GasA,Ex,Y,SBrkr,664,832,0,1496,1,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,2,426,TA,TA,Y,100,24,0,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,968,968,GasA,TA,Y,SBrkr,968,0,0,968,0,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1935.0,Unf,2,480,Fa,TA,N,0,0,184,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609,Unf,0,785,1394,GasA,Gd,Y,SBrkr,1394,0,0,1394,1,0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2,514,TA,TA,Y,0,76,0,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [7]:
test['Alley'].value_counts()

Grvl    35
Pave    23
Name: Alley, dtype: int64

## Data cleaning: Initial check
---

Check the following in the cells below:
1. Do we have any null values?
2. Are any numerical columns being read in as `object`?

In [8]:
test.shape

(879, 80)

In [9]:
#check for nulls
test.isnull().sum()

Id                   0
PID                  0
MS SubClass          0
MS Zoning            0
Lot Frontage       160
Lot Area             0
Street               0
Alley              821
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type         1
Mas Vnr Area         1
Exter Qual           0
Exter Cond           0
Foundation           0
Bsmt Qual           25
Bsmt Cond           25
Bsmt Exposure       25
BsmtFin Type 1      25
BsmtFin SF 1         0
BsmtFin Type 2      25
BsmtFin SF 2         0
Bsmt Unf SF          0
Total Bsmt SF        0
Heating              0
Heating QC           0
Central Air          0
Electrical 

## Data Cleaning 

Filling the null values with the median of the column.
Median is suitable for Continous data with outliers

In [10]:
test.fillna((test[['Lot Frontage','Mas Vnr Area','BsmtFin SF 1','BsmtFin SF 2','Bsmt Unf SF','Total Bsmt SF',
                   'Bsmt Full Bath','Garage Yr Blt','Bsmt Half Bath','Garage Cars','Garage Area' ]].median()), inplace=True)

In [12]:
cols = ['Alley','Mas Vnr Type','Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','BsmtFin Type 2','Fireplace Qu',
    'Garage Type','Garage Finish','Garage Qual','Garage Cond','Pool QC','Fence','Misc Feature','Electrical']
test[cols] = test[cols].fillna(0)

In [13]:
test.isnull().sum().sum()

0

In [14]:
test.dtypes

Id                   int64
PID                  int64
MS SubClass          int64
MS Zoning           object
Lot Frontage       float64
Lot Area             int64
Street              object
Alley               object
Lot Shape           object
Land Contour        object
Utilities           object
Lot Config          object
Land Slope          object
Neighborhood        object
Condition 1         object
Condition 2         object
Bldg Type           object
House Style         object
Overall Qual         int64
Overall Cond         int64
Year Built           int64
Year Remod/Add       int64
Roof Style          object
Roof Matl           object
Exterior 1st        object
Exterior 2nd        object
Mas Vnr Type        object
Mas Vnr Area       float64
Exter Qual          object
Exter Cond          object
Foundation          object
Bsmt Qual           object
Bsmt Cond           object
Bsmt Exposure       object
BsmtFin Type 1      object
BsmtFin SF 1         int64
BsmtFin Type 2      object
B

## Dummies

When we dummy a column, we're converting a categorical column into a one-hot encoded matrix. `pandas` allows us to do this with the `pd.get_dummies()` method.

In [15]:
# created dummies for the nominal data type
test = pd.get_dummies(test, columns = ['MS SubClass','MS Zoning','Street','Alley',
        'Land Contour','Lot Config','Neighborhood','Condition 1','Condition 2',
        'Bldg Type','House Style','Roof Style','Roof Matl','Exterior 1st','Exterior 2nd','Mas Vnr Type',
        'Foundation','Heating','Central Air','Garage Type','Misc Feature','Sale Type'], 
        prefix = ['MS SubClass','MS Zoning','Street','Alley',
        'Land Contour','Lot Config','Neighborhood','Condition 1','Condition 2',
        'Bldg Type','House Style','Roof Style','Roof Matl','Exterior 1st','Exterior 2nd','Mas Vnr Type',
        'Foundation','Heating','Central Air','Garage Type','Misc Feature','Sale Type'],drop_first = True)

## Mapping and Dictionary

Here the columns are  Ordinal. Grouped them in the dictionary using mapping and Dictionary

###  Lot Shape

In [16]:
test['Lot Shape'].value_counts()

Reg    564
IR1    287
IR2     21
IR3      7
Name: Lot Shape, dtype: int64

In [17]:
#mapped using dictionary
test['Lot Shape'] = test['Lot Shape'].map({'Reg': 4, 'IR1': 3, 'IR2' : 2, 'IR3' : 1})

### Utilities

In [18]:
test['Utilities'].value_counts()

AllPub    878
NoSewr      1
Name: Utilities, dtype: int64

In [19]:
#mapped using dictionary
test['Utilities'] = test['Utilities'].map({'AllPub': 3, 'NoSeWa': 2, 'NoSewr' : 1})

### Land Slope

In [20]:
test['Land Slope'].value_counts()

Gtl    836
Mod     37
Sev      6
Name: Land Slope, dtype: int64

In [21]:
#mapped using dictionary
test['Land Slope'] =test['Land Slope'].map({'Gtl': 3, 'Mod': 2, 'Sev' : 1})

### External Quality

In [22]:
test['Exter Qual'].value_counts()

TA    552
Gd    292
Ex     26
Fa      9
Name: Exter Qual, dtype: int64

In [23]:
test['Exter Qual'] = test['Exter Qual'].map({'Ex': 4, 'Gd': 3, 'TA':2, 'Fa': 1})

### External Condition

In [24]:
test['Exter Cond'].value_counts()

TA    771
Gd     84
Fa     18
Ex      5
Po      1
Name: Exter Cond, dtype: int64

In [25]:
test['Exter Cond'] = test['Exter Cond'].map({'Ex': 5, 'Gd': 4, 'TA':3, 'Fa': 2,'Po':1})

### Basement Quality

In [27]:
test['Bsmt Qual'].value_counts()

TA    396
Gd    355
Ex     74
Fa     28
0      25
Po      1
Name: Bsmt Qual, dtype: int64

In [28]:
test['Bsmt Qual'] = test['Bsmt Qual'].map({'Ex': 6, 'Gd': 5, 'TA':4, 'Fa': 3,'Po':2, 0:0})

### Basement Condition

In [29]:
test['Bsmt Cond'].value_counts()

TA    782
Fa     39
Gd     33
0      25
Name: Bsmt Cond, dtype: int64

In [30]:
test['Bsmt Cond'] = test['Bsmt Cond'].map({'Gd': 3, 'TA':2, 'Fa': 1, 0 : 0})

### Basement Exposure

In [31]:
test['Bsmt Exposure'].value_counts()

No    567
Av    130
Gd     81
Mn     76
0      25
Name: Bsmt Exposure, dtype: int64

In [32]:
test['Bsmt Exposure'] =test['Bsmt Exposure'].map({'Gd': 4, 'Av': 3, 'Mn':2, 'No': 1, 0 : 0})

### BasementFin Type 1

In [33]:
test['BsmtFin Type 1'].value_counts()

Unf    248
GLQ    244
ALQ    136
Rec    105
BLQ     69
LwQ     52
0       25
Name: BsmtFin Type 1, dtype: int64

In [34]:
test['BsmtFin Type 1'] = test['BsmtFin Type 1'].map({'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ':2, 'Unf': 1, 0 : 0})

### BasementFin Type 2

In [35]:
test['BsmtFin Type 2'].value_counts()

Unf    750
LwQ     29
Rec     26
0       25
BLQ     20
ALQ     18
GLQ     11
Name: BsmtFin Type 2, dtype: int64

In [36]:
test['BsmtFin Type 2'] = test['BsmtFin Type 2'].map({'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ':2, 'Unf':1, 0 : 0})

### HeatingQC

In [37]:
test['Heating QC'].value_counts()

Ex    430
TA    267
Gd    157
Fa     25
Name: Heating QC, dtype: int64

In [38]:
test['Heating QC'] = test['Heating QC'].map({'Ex': 4, 'Gd': 3, 'TA':2, 'Fa': 1})

### Electrical

In [39]:
test['Electrical'].value_counts()

SBrkr    814
FuseA     48
FuseF     15
FuseP      1
0          1
Name: Electrical, dtype: int64

In [40]:
test['Electrical'] = test['Electrical'].map({'SBrkr': 4, 'FuseA': 3, 'FuseF':2, 'FuseP':1, 0 : 0})

In [41]:
test['Electrical'].isnull().sum()

0

### KitchenQual

In [42]:
test['Kitchen Qual'].isnull().sum()

0

In [43]:
test["Kitchen Qual"].value_counts()

TA    447
Gd    354
Ex     54
Fa     23
Po      1
Name: Kitchen Qual, dtype: int64

In [44]:
test['Kitchen Qual'] = test['Kitchen Qual'].map({'Ex': 5, 'Gd': 4, 'TA':3, 'Fa': 2, 'Po' : 1})

In [45]:
test['Kitchen Qual'].isnull().sum()

0

### Functional

In [46]:
test['Functional'].value_counts()

Typ     813
Min2     28
Min1     23
Maj1      7
Mod       6
Maj2      2
Name: Functional, dtype: int64

In [47]:
test['Functional'] = test['Functional'].map({'Typ': 6, 'Min1': 5, 'Min2':4, 'Mod': 3,'Maj1':2, 'Maj2':1})

### Fireplace Qu

In [48]:
test['Fireplace Qu'].value_counts()

0     422
Gd    221
TA    193
Fa     16
Po     15
Ex     12
Name: Fireplace Qu, dtype: int64

In [49]:
test['Fireplace Qu'] = test['Fireplace Qu'].map({'Ex': 5, 'Gd': 4, 'TA':3, 'Fa': 2,'Po':1, 0 : 0})

### Garage Finish

In [50]:
test['Garage Finish'].value_counts()

Unf    382
RFn    233
Fin    219
0       45
Name: Garage Finish, dtype: int64

In [51]:
test['Garage Finish'] =test['Garage Finish'].map({'Fin': 3, 'RFn': 2, 'Unf':1, 0 : 0})

### Garage Quality

In [52]:
test['Garage Qual'].value_counts()

TA    783
0      45
Fa     42
Gd      6
Po      3
Name: Garage Qual, dtype: int64

In [53]:
test['Garage Qual'] = test['Garage Qual'].map({'Gd': 4, 'TA':3, 'Fa': 2,'Po':1, 0 : 0})

### Garage Cond

In [54]:
test['Garage Cond'].value_counts()

TA    797
0      45
Fa     27
Po      6
Gd      3
Ex      1
Name: Garage Cond, dtype: int64

In [55]:
test['Garage Cond'] = test['Garage Cond'].map({'Ex': 5, 'Gd': 4, 'TA':3, 'Fa': 2,'Po':1, 0 : 0})

### Paved Drive

In [56]:
test['Paved Drive'].value_counts()

Y    791
N     65
P     23
Name: Paved Drive, dtype: int64

In [57]:
test['Paved Drive'] = test['Paved Drive'].map({'Y': 3,'P':2, 'N': 1})

### Pool QC

In [58]:
test['Pool QC'].value_counts()

0     875
Ex      3
TA      1
Name: Pool QC, dtype: int64

In [59]:
test['Pool QC'] = test['Pool QC'].map({'Ex': 2, 'TA':1, 0 : 0})

### Fence

In [60]:
test['Fence'].value_counts()

0        707
MnPrv    103
GdPrv     35
GdWo      32
MnWw       2
Name: Fence, dtype: int64

In [61]:
test['Fence'] = test['Fence'].map({'GdPrv': 4, 'MnPrv':3, 'GdWo': 2,'MnWw':1, 0 : 0})

In [62]:
test.shape

(879, 203)

In [63]:
test.isnull().sum().sum()

0

In [64]:
test.dtypes

Id                        int64
PID                       int64
Lot Frontage            float64
Lot Area                  int64
Lot Shape                 int64
Utilities                 int64
Land Slope                int64
Overall Qual              int64
Overall Cond              int64
Year Built                int64
Year Remod/Add            int64
Mas Vnr Area            float64
Exter Qual                int64
Exter Cond                int64
Bsmt Qual                 int64
Bsmt Cond                 int64
Bsmt Exposure             int64
BsmtFin Type 1            int64
BsmtFin SF 1              int64
BsmtFin Type 2            int64
BsmtFin SF 2              int64
Bsmt Unf SF               int64
Total Bsmt SF             int64
Heating QC                int64
Electrical                int64
1st Flr SF                int64
2nd Flr SF                int64
Low Qual Fin SF           int64
Gr Liv Area               int64
Bsmt Full Bath            int64
Bsmt Half Bath            int64
Full Bat

In [65]:
test.to_csv("test_cleaned.csv")